In [1]:
ls

Introgression.slim  SLiMloop.ipynb


In [4]:
import resource
print(resource.getrlimit(resource.RLIMIT_NOFILE))

(1048576, 9223372036854775807)


In [1]:
import resource
resource.setrlimit(resource.RLIMIT_NOFILE, (1048576, 9223372036854775807))

In [3]:
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed

def parse(out):
    lines = out.split("\n")
    for line in lines:
        if line.startswith("#OUTPUT:"):
            return float(line.split()[1])

def run_slim(run_number, m, g):
    command = f"slim -d runNumber={run_number} -d migrationRate={m} -d g={g} Introgression_ecological_pi2.slim"
    process = subprocess.Popen(
        command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )
    out, err = process.communicate()
    if process.returncode != 0:
        raise RuntimeError(f"Process failed with error: {err}")
    return out

def process_m_and_g(m, g, max_count, num_workers=4, total_limit=100000):
    total = 0  # Number of simulation executions
    count = 0  # Total count of successful cases
    run_number = 0  # Counter to increment dynamically

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_run = {}

        while count < max_count and total < total_limit:
            # Add new tasks as needed
            while len(future_to_run) < num_workers:
                future = executor.submit(run_slim, run_number, m, g)
                future_to_run[future] = run_number
                run_number += 1

            # Process completed tasks
            for future in as_completed(future_to_run):
                try:
                    out = future.result()
                    total += 1
                    count += parse(out)

                    # Remove task from the tracking dictionary
                    del future_to_run[future]

                    # Exit if count reaches max_count
                    if count >= max_count or total >= total_limit:
                        break
                except Exception as e:
                    print(f"Error occurred for migrationRate={m}, growthRate={g}: {e}")
                    del future_to_run[future]

        # Message if total_limit is exceeded
        if total >= total_limit:
            print(f"Limit reached for migrationRate={m}, growthRate={g}: {int(max_count)}/{int(total)} (total limit exceeded)")

    # Normal completion message
    print(f"Done for migrationRate={m}, growthRate={g} with surviving probability: {int(max_count)}/{int(total)}")
    return m, g, total, count

def main():
    m_values = [0.0]  # Values of m for the outer loop
    g_values = [1.2, 1.1, 1.08]  # Values of g for the inner loop
    max_count = 100  # Target value for count

    results = []

    for m in m_values:
        for g in g_values:
            print(f"Processing for migrationRate={m}, growthRate={g}...")
            result = process_m_and_g(m, g, max_count)
            results.append(result)

    # Output statistical information
    print("\n=== Final Statistics ===")
    for m, g, total, count in results:
        print(f"m={m}, g={g} -> Total Simulations: {total}, Successful Count: {count}")
    print("=========================")

if __name__ == "__main__":
    main()

Processing for migrationRate=0.0, growthRate=1.2...
Done for migrationRate=0.0, growthRate=1.2 with surviving probability: 100/120
Processing for migrationRate=0.0, growthRate=1.1...
Done for migrationRate=0.0, growthRate=1.1 with surviving probability: 100/131
Processing for migrationRate=0.0, growthRate=1.08...
Done for migrationRate=0.0, growthRate=1.08 with surviving probability: 100/136

=== Final Statistics ===
m=0.0, g=1.2 -> Total Simulations: 120, Successful Count: 100.0
m=0.0, g=1.1 -> Total Simulations: 131, Successful Count: 100.0
m=0.0, g=1.08 -> Total Simulations: 136, Successful Count: 100.0
